In [ ]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
spark

[Transformer base class](https://github.com/apache/spark/blob/v2.4.0/python/pyspark/ml/base.py#L139)

[Estimator base class](https://github.com/apache/spark/blob/v2.4.0/python/pyspark/ml/base.py#L70)

### Let's create a basic transformer

In [ ]:
from pyspark.ml import Transformer
import pyspark.sql.functions as F

In [ ]:
class ConstTransformer(Transformer):
    """Constant transformer.
    
    It just adds one constant column with a predefined name
    """
    def _transform(self, dataset):
        return dataset.withColumn("mytransformer", F.lit("I am a constant"))

In [ ]:
df = spark.range(0, 10, numPartitions=1)

In [ ]:
transformer = ConstTransformer()

transformer.transform(df).show()

### How do we specify transformer parameters?

[HasOutputCol mixin](https://github.com/apache/spark/blob/v2.4.0/python/pyspark/ml/param/shared.py#L235)

In [ ]:
from pyspark.ml.param.shared import HasOutputCol

In [ ]:
class ConstTransformer(Transformer, HasOutputCol):
    """Constant transformer with variable name."""
    def __init__(self): 
        super(ConstTransformer, self).__init__()
        
    def _transform(self, dataset):
        return dataset.withColumn(self.getOutputCol(), F.lit("I am a constant"))

In [ ]:
transformer = ConstTransformer()

In [ ]:
transformer.extractParamMap()

In [ ]:
transformer.getOutputCol()

In [ ]:
transformer.transform(df).show()

In [ ]:
from pyspark import keyword_only

In [ ]:
class ConstTransformer(Transformer, HasOutputCol):
    @keyword_only
    def __init__(self, outputCol=None): 
        super(ConstTransformer, self).__init__()
        if outputCol is not None:
            self.setOutputCol(outputCol)
        
    def _transform(self, dataset):
        return dataset.withColumn(self.getOutputCol(), F.lit("I am a constant"))

In [ ]:
transformer = ConstTransformer("mycolumn")

In [ ]:
transformer = ConstTransformer(outputCol="myColumn")

In [ ]:
transformer.getOutputCol()

In [ ]:
transformer.transform(df).show()

In [ ]:
transformer.setOutputCol("anotherColumn")

In [ ]:
transformer.transform(df).show()

### Let's define a real-life transformer with input and output columns
[HasInputCol mixin](https://github.com/apache/spark/blob/v2.4.0/python/pyspark/ml/param/shared.py#L189)

In [ ]:
from pyspark.ml.param.shared import HasInputCol

In [ ]:
class HashTransformer(Transformer, HasInputCol, HasOutputCol):
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(HashTransformer, self).__init__()
        if inputCol is not None:
            self.setInputCol(inputCol)
        if outputCol is not None:
            self.setOutputCol(outputCol)
            
    def _transform(self, dataset):
        return dataset.withColumn(self.getOutputCol(), F.md5(F.col(self.getInputCol()).cast("string")))

In [ ]:
transformer = HashTransformer(inputCol="id", outputCol="hash")

In [ ]:
transformer.transform(df).show()

### How do we define a custom parameter?
[Param](https://github.com/apache/spark/blob/v2.4.0/python/pyspark/ml/param/__init__.py#L37)

In [ ]:
from pyspark.ml.param import Param, Params, TypeConverters

In [ ]:
class HashTransformer(Transformer, HasInputCol, HasOutputCol):
    
    algorithm = Param(Params._dummy(), "algorithm",
                      "hash function to use, must be one of (md5|sha1)",
                      typeConverter=TypeConverters.toString)
    
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, algorithm="md5"):
        super(HashTransformer, self).__init__()
        if inputCol is not None:
            self.setInputCol(inputCol)
        if outputCol is not None:
            self.setOutputCol(outputCol)
        self._set(algorithm=algorithm)
        
    def get_hash_function(self):
        try:
            return getattr(F, self.getOrDefault("algorithm"))
        except AttributeError as e:
            raise ValueError("Unsupported algorithm {}".format(algorithm))
            
    def setAlgorithm(self, algorithm):
        self._set(algorithm=algorithm)
            
    def _transform(self, dataset):
        hash_col = self.get_hash_function()
        return dataset.withColumn(self.getOutputCol(), hash_col(F.col(self.getInputCol()).cast("string")))

In [ ]:
transformer = HashTransformer(inputCol="id", outputCol="hash")

In [ ]:
print(transformer.explainParams())

In [ ]:
transformer.getOrDefault("algorithm")

In [ ]:
transformer.transform(df).show()

In [ ]:
transformer.setAlgorithm("sha1")

In [ ]:
transformer.getOrDefault("algorithm")

In [ ]:
transformer.transform(df).show()

In [ ]:
spark.stop()